In [55]:
import requests
import generate
import organize_data
import os

# before start

In [ ]:
import os
from PIL import Image

def resize_and_rename_images(input_path, output_path, prefix):
    # 確保輸出路徑存在
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    # 獲取輸入路徑中的所有檔案
    files = os.listdir(input_path)
    image_files = [f for f in files if f.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'gif', 'webp'))]
    
    for i, file in enumerate(image_files):
        img = Image.open(os.path.join(input_path, file))
        img = img.resize((512, 512), Image.ANTIALIAS)
        new_filename = f"{prefix}{i+1}.png"
        img.save(os.path.join(output_path, new_filename), 'PNG')

# 使用範例
input_path = 'path/to/your/input/directory'
output_path = 'path/to/your/output/directory'
prefix = 'image_'
resize_and_rename_images(input_path, output_path, prefix)

# Utli

In [56]:
def get_queue_amount():
    # Make a GET request to the specified URL
    response = requests.get("http://127.0.0.1:8188/queue")

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        # Calculate the sum of "queue_running" and "queue_pending"
        total = len(data["queue_running"]) + len(data["queue_pending"])
        return total
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")
        return -1


In [57]:
def get_image_names(input_image_prefix = 'DALLE'):
    input_directory = '../../myComfyUI/input'
    image_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.webp')

    # Get all image names in the input directory
    image_names = [f for f in os.listdir(input_directory) if f.lower().endswith(image_extensions)]


    image_names = [name for name in image_names if name.startswith(input_image_prefix)]

    print(image_names)
    return image_names


In [58]:
print(get_image_names())

['DALLEBuilding.png', 'DALLECar.png', 'DALLECar2.png', 'DALLEFish.png', 'DALLEFish.webp', 'DALLEForst.png', 'DALLETree.png']
['DALLEBuilding.png', 'DALLECar.png', 'DALLECar2.png', 'DALLEFish.png', 'DALLEFish.webp', 'DALLEForst.png', 'DALLETree.png']


In [61]:
def generate_target_and_organize(target_image_name, category = 'DALLE'):
    generate.generate_target(target=target_image_name)

    from time import sleep

    while(get_queue_amount() > 0):
        print("not done yet")
        sleep(10)
        
    print("done")
    organize_data.organize_data(folder_name=target_image_name.split('.')[0], category=category)


In [ ]:
def generate_by_category(category):
    image_names = get_image_names()
    for image_name in image_names:
        generate_target_and_organize(image_name, category)

In [ ]:
categories = ["DALLE", "SD"]
for category in categories:
    generate_by_category(category)

18


not done yet
not done yet
not done yet
not done yet
not done yet
not done yet
not done yet
not done yet
not done yet
not done yet
not done yet
not done yet
not done yet
not done yet
not done yet
done
